In [1]:
from selenium import webdriver
import time

In [2]:
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil

# 자신 맞는 chrome.exe 위치 변경 필요
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')

options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

In [3]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../chromedriver.exe', options=options)

In [4]:
# url in address window(주소창에 url 입력)
browser.get('https://www.coupang.com/np/search?q=%EA%B0%84%20%EA%B8%B0%EB%8A%A5%20%EC%98%81%EC%96%91%EC%A0%9C&channel=recent')
browser.implicitly_wait(10)

In [5]:
browser.get('https://www.coupang.com/vp/products/7346382208?itemId=18894168250&vendorItemId=78477362962&sourceType=srp_product_ads&clickEventId=5bb751d0-578d-11ee-b7d6-dd3f93a27cbe&korePlacement=15&koreSubPlacement=1&q=%EA%B0%84+%EA%B8%B0%EB%8A%A5+%EC%98%81%EC%96%91%EC%A0%9C&itemsCount=36&searchId=a4c2c20c6d574c329961fe2bf25caa17&rank=0&isAddedCart=#sdpReview')

In [6]:
import pandas as pd
import pymongo as mg

In [39]:
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [7]:
click_product ='.search-product-wrap'
product_list = browser.find_elements_by_css_selector(click_product)
len(product_list) # 한 페이지당 36개 상품

36

In [8]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['project_nutrients']
collection = database['coupang_liver_review']

column_name = ['product_name', 'review_date', 'review_content', 'review_star', 'review_writer']


i=3  
h=3
review_list = []
review_lists = []
main_window = browser.current_window_handle  # 현재 창의 핸들 저장
from selenium.webdriver.common.keys import Keys
k=2
for go in range(2, 12):
    product_element_path = 'span.btn-page > a:nth-child({})'.format(k)
    product_pagination = browser.find_element_by_css_selector(product_element_path)
    click_product ='.search-product-wrap'
    product_list = browser.find_elements_by_css_selector(click_product)
    for product in product_list:
        product.click()

        browser.implicitly_wait(10)
        print(f"현재 탭 제목: {browser.title}")   
        time.sleep(2)

        # 새로운 탭이 열릴 때까지 대기 (예시: 2개 이상의 창이 열리고, 그 중 하나가 현재 탬과 다른 창일 때까지 대기)
        wait = WebDriverWait(browser, 10)
        new_tab_handle = wait.until(EC.new_window_is_opened)

        # 모든 탬의 핸들 가져오기
        all_tab_handles = browser.window_handles

        # 새로운 탬의 핸들 찾아 선택하기
        for handle in all_tab_handles:
            if handle != main_window:
                new_tab_handle = handle

        # 새로운 탬으로 전환하기
        browser.switch_to.window(new_tab_handle)

        # 필요한 작업 수행 및 정보 가져오기 (예시: 페이지 제목 가져오기)
        print(f"새로운 탭 제목: {browser.title}")   

        try :
            # 상품평 클릭
            browser.find_element_by_css_selector('#prod-review-nav-link').send_keys(Keys.ENTER)
            page = browser.find_element_by_css_selector('.product-tab-review-count').text
            page = page.replace(",", "")  # 쉼표 제거
            page = page.replace("(", "")  # 왼쪽 괄호 제거
            page = page.replace(")", "")  # 오른쪽 괄호 제거

            # 이제 정수로 변환 시도
            try:
                page_int = int(page)
                review_page = page_int / 5
                print(review_page)
            except ValueError as e:
                print("타입 변환 오류:", e)


            for go in range(1, page_int, 1):
            # for go in range(1, 2 ,1):
                element_path = 'div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child({})'.format(h)
                pagination = browser.find_element_by_css_selector(element_path)
                try:
                    product = browser.find_element_by_css_selector(' article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info__name'.format(i)).text
                except:
                    product = str()
                try:
                    star = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__star-gray > div'.format(i)).get_attribute('data-rating')
                except:
                    star = str() 
                try:
                    date = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date'.format(i)).text
                except:
                    date = str()
                try:
                    content = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__review.js_reviewArticleContentContainer > div'.format(i)).text
                except:
                    content = str()
                try:
                    writer = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__user > span'.format(i)).text
                except:
                    writer = str()

                review_list= [product, date, content, star, writer]
                review_lists.append(review_list)
                
                # print(star)
                i = i + 1        
                if i == 8 :
                    pagination.click()
                    time.sleep(1)
                    h = h + 1
                    i = 3

                if h == 13:  # h 값이 12일 때 다시 3으로 초기화
                    h = 3

                    print('실패: {}'.format(element_path))
                    break
            # 저장을 위한 작업 
            df_coupang_liver = pd.DataFrame(data=review_lists, columns=column_name)
            data_dict = df_coupang_liver.to_dict(orient='records')
            collection.insert_many(data_dict)
            review_lists = list()

        except :
            pass

        time.sleep(3)
        # 다시 메인 창으로 전환
        browser.switch_to.window(main_window)
        time.sleep(1)
    product_pagination.click()
    k=k+1
    time.sleep(1)



product_list[0].click()



현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 고려은단 퓨어 밀크씨슬 60캡슐 60정, 3개
711.2
실패: div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child(12)
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 대웅생명과학 가네굿 프리미엄 진 밀크씨슬 세트
389.8
실패: div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child(12)
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 종근당 간건강 헬씨칸 로얄 피로회복 밀크씨슬800mgX30캡슐(2달분)
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 닥터슈퍼칸 간영양제 밀크시슬 밀크씨슬 실리마린 30정, 6개
374.2
실패: div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child(12)
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 엉겅퀴 실리마린 밀크씨슬 추출물 간 건강 기능 식품 보조제 개선제 영양제 셀레늄 비타민 B1 B2 B6 간영양제 간에 좋은 영양제 30대 40대 50대 시니어 선물 판토텐산 식약처인증
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 40대 남자 간기능 개선제 고려은단 밀크시슬 실리마린 퓨어 밀크씨슬 간에좋은 영양제 간 건강 기능 식품 엉겅퀴 추출물 밀크시스 30대 50대 60대 중년 시니어 여성 남성 추천
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | (6개월분)대웅생명과학 가네굿 프리미엄 진 밀크씨슬 30정X6개
7.0
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 에이치엘 사이언스 닥터슈퍼칸에이엘사이언스 30정,

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.btn-page > a:nth-child(10)"}
  (Session info: chrome=116.0.5845.188)


In [257]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [39]:
list = browser.find_element_by_css_selector('#productList').text
print(list)

무료배송
고려은단 퓨어 밀크씨슬 60캡슐, 60정, 1개
44% 25,000
13,900원
(1정당 232원)
10/6 도착 예정
5.0
(3538)
최대 695원 적립
AD
무료배송
트루포뮬러 칸톱 밀크씨슬 3병(3개월) (실리마린 간에좋은영양제), 60정, 3개
67% 264,490
85,900원
(1정당 477원)
9/26(화) 도착 예정
4.5
(3453)
최대 4,295원 적립
1
트루포뮬러 칸톱 밀크씨슬, 60정, 1개
7% 31,330
28,890원
(1정당 482원)
내일(토) 도착 보장
4.5
(3453)
최대 1,445원 적립
2
대웅제약 에너씨슬 테아닌 B+ 48g, 60정, 1개
34,000원
(1정당 567원)
내일(토) 도착 보장
5.0
(5)
최대 1,700원 적립
3
무료배송
고려은단 퓨어 밀크씨슬 60캡슐, 60정, 3개
52% 75,000
35,900원
(1정당 199원)
10/6 도착 예정
5.0
(3538)
최대 1,795원 적립
AD
종근당 간건강 헬씨칸 로얄 피로회복 밀크씨슬800mgX30캡슐(2달분)
7% 28,500
26,500원
10/4 도착 예정
최대 1,325원 적립
AD
무료배송
[한미양행]간에는 밀크씨슬 900mg 2박스 (6개월분)+쇼핑백, 단일속성, 2개
5% 24,900
23,410원
10/4 도착 예정
5.0
(39)
최대 1,170원 적립
4
안국건강 간에 좋은 안국 밀크씨슬, 60정, 48g, 5개
64% 150,000
53,550원
(1정당 179원)
내일(토) 도착 보장
5.0
(1532)
최대 2,678원 적립
5
무료배송
고려은단 퓨어 밀크씨슬 60캡슐, 60정, 2개
48% 50,000
25,900원
(1정당 216원)
10/6 도착 예정
5.0
(3538)
최대 1,295원 적립
AD
무료배송
대웅제약 에너씨슬 퍼펙트샷 10개입X2박스 (20개), 10정, 2개
84,290원
(1정당 4,215원)
9/26(화) 도착 예정
5.0
(7)
최대 4,215원 적립
6
종근당 간건강 헬씨칸 로

In [19]:
page=browser.find_element_by_css_selector('.product-tab-review-count').text
page

'(3,538)'

In [259]:
page = page.replace(",", "")  # 쉼표 제거
page = page.replace("(", "")  # 왼쪽 괄호 제거
page = page.replace(")", "")  # 오른쪽 괄호 제거

# 이제 정수로 변환 시도
try:
    page_int = int(page)
    review_page = page_int / 5
    print(review_page)
except ValueError as e:
    print("타입 변환 오류:", e)

373.4


In [260]:
# review_list = []
review_lists = []

#for i in range(3, 8):  # i 값을 3부터 7까지 변경

i=3  
h=3
for go in range(1, page_int, 1):
    element_path = 'div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child({})'.format(h)
    pagination = browser.find_element_by_css_selector(element_path)
    try:
        star = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__star-gray > div'.format(i)).get_attribute('data-rating')
        date = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date'.format(i)).text
        content = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__review.js_reviewArticleContentContainer > div'.format(i)).text
        writing = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__user > span'.format(i)).text
        review_list= [writing, date, star, content]
        review_lists.append(review_list)
        
        print(star)
        i = i + 1        
        if i == 8 :
            pagination.click()
            time.sleep(1)
            h = h + 1
            i = 3

        if h == 13:  # h 값이 12일 때 다시 3으로 초기화
            h = 3
       
        

    except:
        print('실패: {}'.format(element_path))
        break

        
review_lists

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child(3)"}
  (Session info: chrome=116.0.5845.188)


In [22]:
type(page_int)

int

In [8]:
star = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__star-gray > div'.format(i)).get_attribute('data-rating')

star

NameError: name 'i' is not defined

In [251]:
# 페이지 네이션 코드 연습
h=3
for go in range(1, 5, 1):
    element_path = 'div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child({})'.format(h)
    try :
        pagination = browser.find_element_by_css_selector(element_path)
        pagination.click()
        h=h+1
        if h==13 :
            h = 3
        time.sleep(3)
    except : 
        print('실패: {}'.format(element_path))
        break


실패: div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child(3)


In [125]:
# 날짜 코드 테스트
date = browser.find_element_by_css_selector('article:nth-child(3) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date').text
date

'2023.06.27'

In [127]:
# 내용 코드 테스트
content = browser.find_element_by_css_selector('article:nth-child(3) > div.sdp-review__article__list__review.js_reviewArticleContentContainer > div').text
content

'이런분들 드세요~\n●평소 간 건강이 염려되시는 분\n●회식과 야근이 잦은 분\n●간을 건강하게 유지하고 싶은 분\n●불규칙한 식습관을 가지고 계신 분\n\n술을 자주 마시는 분들도~\n술을 마시지 않는 분들도~\n간의 피로는 잘 풀어야 해요~\n밀크시슬로 미리미리 간의 피로를 풀어야겠어요~^^'

In [258]:
page = browser.find_element_by_css_selector('.product-tab-review-count').text
page = page.replace(",", "")  # 쉼표 제거
page = page.replace("(", "")  # 왼쪽 괄호 제거
page = page.replace(")", "")  # 오른쪽 괄호 제거

# 이제 정수로 변환 시도
try:
    page_int = int(page)
    print(page_int)
except ValueError as e:
    print("타입 변환 오류:", e)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".product-tab-review-count"}
  (Session info: chrome=116.0.5845.188)


In [252]:
# 별점 테스트 코드
review_list = []

i=3
for go in range(1, page_int, 1):
    # star = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__star-gray > div'.format(i)).get_attribute('data-rating')
    # date = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date'.format(i)).text
    content = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__review.js_reviewArticleContentContainer > div'.format(i)).text
    writing = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__user > span'.format(i)).text
    review_list.append([writing, date, star, content])
    i=i+1
    if i == 8 :
        i = 3
review_list

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"article:nth-child(3) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__star-gray > div"}
  (Session info: chrome=116.0.5845.188)


In [ ]:
#browser.quitit()

## 상품 클릭 강사님 코드

In [73]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

main_window = browser.current_window_handle  # 현재 창의 핸들 저장
from selenium.webdriver.common.keys import Keys
k=2
for go in range(2, 11):
    product_element_path = 'span.btn-page > a:nth-child({})'.format(k)
    product_pagination = browser.find_element_by_css_selector(product_element_path)
    click_product ='.search-product-wrap'
    product_list = browser.find_elements_by_css_selector(click_product)
    for product in product_list:
        product.click()

        browser.implicitly_wait(10)
        print(f"현재 탭 제목: {browser.title}")   
        time.sleep(2)

        # 새로운 탭이 열릴 때까지 대기 (예시: 2개 이상의 창이 열리고, 그 중 하나가 현재 탬과 다른 창일 때까지 대기)
        wait = WebDriverWait(browser, 10)
        new_tab_handle = wait.until(EC.new_window_is_opened)

        # 모든 탬의 핸들 가져오기
        all_tab_handles = browser.window_handles

        # 새로운 탬의 핸들 찾아 선택하기
        for handle in all_tab_handles:
            if handle != main_window:
                new_tab_handle = handle

        # 새로운 탬으로 전환하기
        browser.switch_to.window(new_tab_handle)

        # 필요한 작업 수행 및 정보 가져오기 (예시: 페이지 제목 가져오기)
        print(f"새로운 탭 제목: {browser.title}")   

        try :
            # 상품평 클릭
            browser.find_element_by_css_selector('#prod-review-nav-link').send_keys(Keys.ENTER)

            # 상품 댓글 
            comment_length = len(browser.find_elements_by_css_selector('section.js_reviewArticleListContainer > article'))
            print(f'comment_length : {comment_length}')
        except :
            pass

        time.sleep(3)
        # 다시 메인 창으로 전환
        browser.switch_to.window(main_window)
        time.sleep(2)
    product_pagination.click()
    k=k+1
    time.sleep(1)

# In[10]:


product_list[0].click()


현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 고려은단 퓨어 밀크씨슬 60캡슐 60정, 1개
comment_length : 5
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 닥터슈퍼칸 간영양제 밀크시슬 밀크씨슬 실리마린 30정, 6개
comment_length : 5
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 솔가 밀크씨슬 130 27g
comment_length : 5
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 트루포뮬러 칸톱 밀크씨슬 2병(2개월) (실리마린 간에좋은영양제) 60정, 2개
comment_length : 5
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 종근당 간건강 헬씨칸 로얄 피로회복 밀크씨슬800mgX30캡슐(2달분)
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 종근당 남성 여성 올인원 영양제 간 건강 기능 식품 보조제 개선제 아연 비타민 B1 B2 b6 나이아신 면역력 기능 간에좋은영양제 식약처인증 홈쇼핑 20대 30대 40대 남자 여자
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 한미양행 헤파케어골드 1000MG x 180캡슐 간건강 간영양제 밀크씨슬 코스트코
comment_length : 5
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | [본사직영] 대웅제약 에너씨슬 콜레다운 밀크씨슬 홍국
comment_length : 5
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 고려은단 퓨어 밀크씨슬 60캡슐 60정, 2개
comment_length : 5
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 쏜리서치 실리포스 베지테리안 캡슐 90정, 1개
comment_length : 5
현재 탭 제목: 쿠팡! - 간 기능 영양제
새로운 탭 제목: 쿠팡! | 대용량 100캡슐 실리마린밀크시

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.188)
